In [1]:
from database.models import HistoricalOHLC
from datetime import date, datetime
from dateutil.relativedelta import relativedelta
import tools.tools as tools
import requests
import json

### Inicialização do Banco SQLite

In [2]:
historical = HistoricalOHLC()
historical.drop_table()
historical.create_table()

### Extração de Dados da API OHLC (Open, High, Low, Close) e Inserção no Banco

In [6]:
#Configs
today = date.today()
dt_format = "%Y-%m-%d"

BASE_OHLC_URL = 'https://api.coinmarketcap.com/data-api/v3.1/cryptocurrency/historical'

for i in range(0,60):
    
    date_ini = today - relativedelta(months=i+1)
    date_end = today - relativedelta(months=i)
    start_unixtime = tools.to_unixtime_sec(date_ini.strftime(dt_format))
    end_unixtime = tools.to_unixtime_sec(date_end.strftime(dt_format))

    #Extração 
    param = f'?id=1&convertId=2783&timeStart={start_unixtime}&timeEnd={end_unixtime}&interval=1d'
    response = requests.get(BASE_OHLC_URL + param)
    items = response.json()['data']['quotes']

    for item in items:
        #Inserção
        historical.insert((
            item['timeOpen'][0:10],
            item['quote']['open'],
            item['quote']['high'],
            item['quote']['low'],
            item['quote']['close']
        ))


### Extração dos Dados Formatados do Banco de Dados

In [24]:
today = date.today()
dt_format = "%Y-%m-%d"

date_1month = today - relativedelta(months=-1)
date_5year = today - relativedelta(years=-5)

graph_candle = historical.read_with_filter(f'1=1')
print(graph_candle)

graph_all = historical.read_with_filter(f"1=1")
print(graph_all)

[(1, '2023-07-24', 143681.4771275978, 143737.5165083175, 136977.8419840203, 137928.0325714501, '2023-08-23 18:33:18.639048'), (2, '2023-07-25', 137937.7450918009, 139386.4208212477, 137383.9393903729, 138841.7972560926, '2023-08-23 18:33:18.736027'), (3, '2023-07-26', 138834.0508518969, 140543.4171960038, 138305.6357426655, 139075.0550284948, '2023-08-23 18:33:18.860412'), (4, '2023-07-27', 139069.4927131744, 140062.8142459477, 137936.7495408185, 138549.2203113384, '2023-08-23 18:33:18.983891'), (5, '2023-07-28', 138556.216502844, 139725.3273265162, 138149.7108126874, 138721.0776999183, '2023-08-23 18:33:19.106555'), (6, '2023-07-29', 138722.0269826859, 139088.2265385317, 138460.4763083315, 138899.3243686571, '2023-08-23 18:33:19.222565'), (7, '2023-07-30', 138900.1528643469, 139307.4121239726, 137299.6696509093, 138319.9749875003, '2023-08-23 18:33:19.332285'), (8, '2023-07-31', 138334.1782440988, 139333.0730630521, 137798.1951455535, 138144.4270235809, '2023-08-23 18:33:19.470368'), 

### Gráficos